<a href="https://colab.research.google.com/github/jesung/Tic-Tac-Toe/blob/master/tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pprint
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random

model = keras.models.Sequential([
    keras.layers.Dense(18, activation=tf.nn.relu, input_shape=(1,9)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(9, activation=tf.nn.relu),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])

class tic_tac_toe:
    def __init__(self):
        self.board = np.array([[2,2,2],[2,2,2],[2,2,2]])
        self.turn = 0

    def reset(self):
        self.board = np.array([[2,2,2],[2,2,2],[2,2,2]])
        self.turn = 0

    def move(self,x,y):
        ##### need to update #######
        if False:   
            print('Invalid move - game is over')
        else:
            self.board[x,y] = self.turn
            
            if self.turn == 0:
                self.turn = 1
            else:
                self.turn = 0

    def win(self):
        for i in range(3):
            if self.board[0,i]==self.board[1,i]==self.board[2,i]<2:
                return 'Won'
            if self.board[i,0]==self.board[i,1]==self.board[i,2]<2:
                return 'Won'

        if self.board[0,0]==self.board[1,1]==self.board[2,2]<2:
            return 'Won'
        if self.board[0,2]==self.board[1,1]==self.board[0,2]<2:
            return 'Won'

        if not 2 in self.board:
            return 'Drawn'
        else:
            return 'In-Progress'

def move_gen(board_state, turn):
    legal_moves = {}
    
    for i in range(3):
        for j in range(3):
            if board_state[i,j] == 2:
                tmp_board = board_state.copy()
                tmp_board[i,j] = turn
                legal_moves[(i,j)]=tmp_board.flatten()

    return legal_moves
  
def move_sel(move_list):
    tmp = np.zeros((9,3))
    i = 0

    for key, value in move_list.items():
      tmp[i,] = [key[0],key[1],model.predict(value.reshape(1,1,9))[0][0][0]]
      i = i + 1

    j = random.choices(range(9),tmp[:,2])
    return (tmp[j,0][0],tmp[j,1][0])


In [119]:
#model.summary()

game = tic_tac_toe()
game.move(1,1)

#print(game.board)
#pprint.pprint(move_gen(game.board,game.turn))

move_sel(move_gen(game.board,game.turn))



(1.0, 0.0)